In [1]:
%matplotlib nbagg
import os
#from time import sleep
import datetime
import time

import matplotlib.pyplot as plt
import numpy as np
import qcodes as qc
import math as math
from qcodes import (
    Measurement,
    experiments,
    initialise_database,
    initialise_or_create_database_at,
    load_by_guid,
    load_by_run_spec,
    load_experiment,
    load_last_experiment,
    load_or_create_experiment,
    new_experiment,
)
from qcodes.dataset.plotting import plot_dataset
from qcodes.logger.logger import start_all_logging
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372


print ("Done")

Done


In [2]:
start_all_logging()
station = qc.Station() # we will load all instruments
print ("Done")

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Lab\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Lab\.qcodes\logs\211013-15832-qcodes.log
Done


In [3]:
lsA = Model_372('lakeshore_372', 'TCPIP::192.168.0.114::7777::SOCKET') #Ask
h = lsA.sample_heater
station.add_component(lsA)
station.add_component(h)

Connected to: LSCI 372 (serial:LSA283C, firmware:1.4) in 0.05s


'lakeshore_372_sample_heater'

In [4]:
lsA.ch01.units('kelvin'); lsA.ch01.enabled(True)
lsA.ch02.units('kelvin'); lsA.ch02.enabled(True)
lsA.ch03.units('kelvin'); lsA.ch03.enabled(True)
lsA.ch05.units('kelvin'); lsA.ch05.enabled(True)
lsA.ch06.units('kelvin'); lsA.ch06.enabled(True)
lsA.ch09.units('kelvin'); lsA.ch09.enabled(True)

In [5]:
for ch in lsA.channels:
    print(f'Temperature of {ch.short_name} ({"on" if ch.enabled() else "off"}): {ch.temperature()} {ch.units()}')

Temperature of ch01 (on): 51.0733 kelvin
Temperature of ch02 (on): 3.00307 kelvin
Temperature of ch03 (on): 3.59782 kelvin
Temperature of ch04 (off): 0.0 ohms
Temperature of ch05 (on): 0.92363 kelvin
Temperature of ch06 (on): 0.0099577 kelvin
Temperature of ch07 (off): 0.0 ohms
Temperature of ch08 (off): 0.0 ohms
Temperature of ch09 (on): 0.0191596 kelvin
Temperature of ch10 (off): 0.0 ohms
Temperature of ch11 (off): 0.0 ohms
Temperature of ch12 (off): 0.0 ohms
Temperature of ch13 (off): 0.0 ohms
Temperature of ch14 (off): 0.0 ohms
Temperature of ch15 (off): 0.0 ohms
Temperature of ch16 (off): 0.0 ohms


In [9]:
# pedagogical coding stuff
print(list(h.RANGES.keys()))   #give me all the available currents
h.mode('open_loop')            #set the heater to open-loop
h.output_range('100μA')        #select the range
h.write("MOUT0.0\n")           #write current level to the heater

['off', '31.6μA', '100μA', '316μA', '1mA', '3.16mA', '10mA', '31.6mA', '100mA']


In [10]:
#do a sanity check
print(h.output_range())
print(h.output())
print(lsA.ch09.temperature())

100μA
0.0
0.0190205


In [11]:
number = 0

In [12]:
number += 1
datafile_path = 'temperature_measurement_T(P)_03_'+str(datetime.date.today())+'-Sweep0'+str(number)+'.txt'
header = "Start time is "+datetime.datetime.now().strftime("%X")+"\n"
with open(datafile_path, 'w+') as f:
    np.savetxt(f, [], header=header)
    header = "Heater-output, Time, TempMXC, TempProbe\n"+"% ,s, K ,K \n"+"Comments: n.a."
    np.savetxt(f, [], header=header)
    
    
lsA.ch01.units('kelvin'); lsA.ch01.enabled(False)
lsA.ch02.units('kelvin'); lsA.ch02.enabled(False)
lsA.ch03.units('kelvin'); lsA.ch03.enabled(False)
lsA.ch05.units('kelvin'); lsA.ch05.enabled(False)
lsA.ch06.units('kelvin'); lsA.ch06.enabled(True)
lsA.ch09.units('kelvin'); lsA.ch09.enabled(True)

h.output_range('100μA')
h.write("MOUT20.0\n")

while True:
    t_time           = time.time()     # log the time
    heateroutput1    = (h.output())    # log the set output power
    temperatureMXC   = (lsA.ch06.temperature()) #log the MXC temperature
    temperatureProbe = (lsA.ch09.temperature()) #log the probe temperature 
    
    #write data to our file
    data = np.column_stack((heateroutput1,t_time,temperatureMXC,temperatureProbe))
    with open(datafile_path, 'a+') as f:
        np.savetxt(f, data)
        f.flush()
        
    time.sleep(10)


KeyboardInterrupt: 

In [38]:
h.write("MOUT54.77\n")

In [23]:
# We start the modifications here 

Temp_mk = [9.125,18.25,37.5,75,150,300]
Temp_mk = np.array(Temp_mk)

T = np.float64(Temp_mk)
I = np.sqrt(0.04/120)*T


print(I*1e3)

[ 166.59894457  333.19788915  684.65319688 1369.30639376 2738.61278753
 5477.22557505]


In [40]:
number += 1
datafile_path = 'temperature_measurement_T(P)_03_'+str(datetime.date.today())+'-Sweep1'+str(number)+'.txt'
header = "Start time is "+datetime.datetime.now().strftime("%X")+"\n"
with open(datafile_path, 'w+') as f:
    np.savetxt(f, [], header=header)
    header = "Heater-output, Time, TempMXC, TempProbe\n"+"% ,s, K ,K \n"+"Comments: Exercise 3, step alike" 
    np.savetxt(f, [], header=header)
    
    
lsA.ch01.units('kelvin'); lsA.ch01.enabled(False)
lsA.ch02.units('kelvin'); lsA.ch02.enabled(False)
lsA.ch03.units('kelvin'); lsA.ch03.enabled(False)
lsA.ch05.units('kelvin'); lsA.ch05.enabled(False)
lsA.ch06.units('kelvin'); lsA.ch06.enabled(True)
lsA.ch09.units('kelvin'); lsA.ch09.enabled(True)

h.output_range('100μA')
h.write("MOUT0.0\n")

while True:
    t_time           = time.time()     # log the time
    heateroutput1    = (h.output())    # log the set output power
    temperatureMXC   = (lsA.ch06.temperature()) #log the MXC temperature
    temperatureProbe = (lsA.ch09.temperature()) #log the probe temperature 
    
    #write data to our file
    data = np.column_stack((heateroutput1,t_time,temperatureMXC,temperatureProbe))
    with open(datafile_path, 'a+') as f:
        np.savetxt(f, data)
        f.flush()
        
    time.sleep(10)


KeyboardInterrupt: 

In [32]:
Temp_mk = [37.5,75,150,300]
Temp_mk = np.array(Temp_mk)

T = np.float64(Temp_mk)
I = np.sqrt(0.04/120)*T

values = (I*1e3)*0.1 
values =  str(np.round(values, 2))
variable = "MOUT"+str(values)+"\n"

print(variable)


MOUT[ 68.47 136.93 273.86 547.72]

